In [ ]:
'''
ArrNet iplements a TCN architecture to create a model that
refines onsite time estimation on seismic waveform picks.

This notebook is a releasable work in progress that shows how to
instantiate the model and perform a 'dummy' inference with the untrained model.

Note that ArrNet is a fully convolutional network capable of accepting any 
input size.  Care must be taken that the desired features to be excratcted 
fit within the network's receptive field.  The default settings on this 
example have a receptive field of ~30 seconds.
'''

In [ ]:
# If you have downloaded model_utils.py and ArrNet.py to this working directory,
# this line will install them for you.
!pip install -e .

Obtaining file:///content
  Found existing installation: sdea 0.0.0
    Can't uninstall 'sdea'. No files were found to uninstall.
  Running setup.py develop for sdea


In [ ]:
import os
import numpy as np
import tensorflow as tf

from ArrNet import ArrNet
from model_utils import Params
from model_utils import save_dict_to_json

In [ ]:
# Call the get_default_par method to save a set of default parameters in a json file.
# All hyperparameters for the network are controlled by setting values in this par file.

params = ArrNet.get_default_par(os.path.join('.', 'test_par.json'))
params

lr             : 0.001
bs             : 512
loss           : quantile
optimizer      : adam
f_1ow          : 0.8
f_high         : 4.5
f_pad          : 3
s_rate         : 40
w_len          : 30
shift          : 0
f              : 45
d              : [2, 4, 8, 16, 32]
k              : 20
s              : 5
dense          : [64, 64]
pat            : 20
t_step         : 1000
v_step         : 60
cmps           : B
project_name   : 
model_name     : lr:0.001|bs:512|loss:quantile|optimizer:adam|f_1ow:0.8|f_high:4.5|f_pad:3|s_rate:40|w_len:30|shift:0|f:45|d:2x4x8x16x32|k:20|s:5|dense:64x64|pat:20|t_step:1000|v_step:60|cmps:B|trnRET:False|type:tcn|quantiles:0.9772x0.8413x0.1538x0.0228
model_file     : 
model_save     : 
log_folder     : 
data_folder    : 
image_folder   : 
model_folder   : 
trnRET         : False
catalog        : 
type           : tcn
quantiles      : [0.9772, 0.8413, 0.1538, 0.0228]



In [ ]:
# Create a model using the previously created params.
# Settings can be adjusted in the parameters by setting the values
# Ex. params.lr = 0.0001
model = ArrNet.get_network(params)
model.summary()

Model: "q_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, 1)]    0                                            
__________________________________________________________________________________________________
conv1d_50 (Conv1D)              (None, None, 45)     945         input_2[0][0]                    
__________________________________________________________________________________________________
activation_25 (Activation)      (None, None, 45)     0           conv1d_50[0][0]                  
__________________________________________________________________________________________________
spatial_dropout1d_25 (SpatialDr (None, None, 45)     0           activation_25[0][0]              
____________________________________________________________________________________________

In [ ]:
# With the model created, lets make a fake data trace to test it
data = np.random.uniform(size=(1,1200,1))

In [ ]:
# The output of this inference corresponds to [0] = y_hat, [1..n] = the chosen percentiles
model(data)

[<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-4.245496]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-9.755591]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[4.808104]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-10.12789]], dtype=float32)>,
 <tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-10.675251]], dtype=float32)>]